# Project 3: Big graphs

The objective of this project is to use Spark’s APIs to analyze the flight interconnected data to understand the popularity of the airports and flight patterns.

## Task 1: Data Ingestion and Preparation

### Session Setup

In [1]:
import pyspark

# Prepare the Spark builder
spark = pyspark.sql.SparkSession.builder.appName("Project_4") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "16g") \
    .getOrCreate()

spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
spark.conf.set("spark.sql.shuffle.partitions", spark._sc.defaultParallelism)
spark.conf.set("spark.sql.repl.eagerEval.truncate", 500)

In [2]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

### Data Ingestion

In [3]:
schema = StructType([
    StructField("FL_DATE", DateType(), True),
    StructField("OP_CARRIER", StringType(), True),
    StructField("OP_CARRIER_FL_NUM", IntegerType(), True),
    StructField("ORIGIN", StringType(), True),
    StructField("DEST", StringType(), True),
    StructField("CRS_DEP_TIME", IntegerType(), True),
    StructField("DEP_TIME", DoubleType(), True),
    StructField("DEP_DELAY", DoubleType(), True),
    StructField("TAXI_OUT", DoubleType(), True),
    StructField("WHEELS_OFF", DoubleType(), True),
    StructField("WHEELS_ON", DoubleType(), True),
    StructField("TAXI_IN", DoubleType(), True),
    StructField("CRS_ARR_TIME", IntegerType(), True),
    StructField("ARR_TIME", DoubleType(), True),
    StructField("ARR_DELAY", DoubleType(), True),
    StructField("CANCELLED", DoubleType(), True),
    StructField("CANCELLATION_CODE", StringType(), True),
    StructField("DIVERTED", DoubleType(), True),
    StructField("CRS_ELAPSED_TIME", DoubleType(), True),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), True),
    StructField("AIR_TIME", DoubleType(), True),
    StructField("DISTANCE", DoubleType(), True),
    StructField("CARRIER_DELAY", DoubleType(), True),
    StructField("WEATHER_DELAY", DoubleType(), True),
    StructField("NAS_DELAY", DoubleType(), True),
    StructField("SECURITY_DELAY", DoubleType(), True),
    StructField("LATE_AIRCRAFT_DELAY", DoubleType(), True),
    StructField("Unnamed: 27", StringType(), True)
])

# Reading in the datasets
flight_df = spark.read.csv("input/2009.csv", header=True, schema=schema)
test_df = spark.read.csv("input/2010.csv", header=True, schema=schema)

flight_df.cache()

DataFrame[FL_DATE: date, OP_CARRIER: string, OP_CARRIER_FL_NUM: int, ORIGIN: string, DEST: string, CRS_DEP_TIME: int, DEP_TIME: double, DEP_DELAY: double, TAXI_OUT: double, WHEELS_OFF: double, WHEELS_ON: double, TAXI_IN: double, CRS_ARR_TIME: int, ARR_TIME: double, ARR_DELAY: double, CANCELLED: double, CANCELLATION_CODE: string, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, DISTANCE: double, CARRIER_DELAY: double, WEATHER_DELAY: double, NAS_DELAY: double, SECURITY_DELAY: double, LATE_AIRCRAFT_DELAY: double, Unnamed: 27: string]

### Partitioned Parquet

TBA

## Task 2: Cleaning and Preprocessing 

### Renaming columns for consistency

In [4]:
renamed_columns = [
    "Date", "UniqueCarrier", "FlightNumber", "Origin", "Destination",
    "CRSDepTime", "DepartureTime", "DepartureDelay", "TaxiOut", "WheelsOff",
    "WheelsOn", "TaxiIn", "CRSArrivalTime", "ArrivalTime", "ArrivalDelay",
    "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime",
    "ActualElapsedTime", "AirTime", "Distance", "CarrierDelay",
    "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay",
    "UnusedColumn"
]

flight_df = flight_df.toDF(*renamed_columns)
test_df = test_df.toDF(*renamed_columns)

### Creating day of week and month columns

In [5]:
flight_df = flight_df.withColumn("DayofWeek", F.dayofweek("Date")) \
                     .withColumn("Month", F.month("Date"))

test_df = test_df.withColumn("DayofWeek", F.dayofweek("Date")) \
                     .withColumn("Month", F.month("Date"))

### Handindling empty values

As the goal of the model is to predict flight cancellation, only attributes that relate to events which have happened before the departure/ cancellation can be used for predictions. Therefore, columns such as WheelsOff, ArrivalTime etc. are removed from the training data.

In [6]:
# Empty values in dataset
flight_df.select([
    F.count(F.when(F.col(c).isNull() | (F.isnan(c) if dict(flight_df.dtypes)[c] in ('double', 'float') else F.lit(False)), c)).alias(c)
    for c in flight_df.columns
]).show()

+----+-------------+------------+------+-----------+----------+-------------+--------------+-------+---------+--------+------+--------------+-----------+------------+---------+----------------+--------+--------------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+------------+---------+-----+
|Date|UniqueCarrier|FlightNumber|Origin|Destination|CRSDepTime|DepartureTime|DepartureDelay|TaxiOut|WheelsOff|WheelsOn|TaxiIn|CRSArrivalTime|ArrivalTime|ArrivalDelay|Cancelled|CancellationCode|Diverted|CRSElapsedTime|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|UnusedColumn|DayofWeek|Month|
+----+-------------+------------+------+-----------+----------+-------------+--------------+-------+---------+--------+------+--------------+-----------+------------+---------+----------------+--------+--------------+-----------------+-------+--------+------------+------------+--------+-------------+-

In [7]:
flight_df_c = flight_df.filter(F.col("Cancelled") == 1)

print("Cancelled flights:", flight_df_c.count())

flight_df_c.select([
    F.count(F.when(F.col(c).isNull() | (F.isnan(c) if dict(flight_df.dtypes)[c] in ('double', 'float') else F.lit(False)), c)).alias(c)
    for c in flight_df_c.columns
]).show()

Cancelled flights: 87038
+----+-------------+------------+------+-----------+----------+-------------+--------------+-------+---------+--------+------+--------------+-----------+------------+---------+----------------+--------+--------------+-----------------+-------+--------+------------+------------+--------+-------------+-----------------+------------+---------+-----+
|Date|UniqueCarrier|FlightNumber|Origin|Destination|CRSDepTime|DepartureTime|DepartureDelay|TaxiOut|WheelsOff|WheelsOn|TaxiIn|CRSArrivalTime|ArrivalTime|ArrivalDelay|Cancelled|CancellationCode|Diverted|CRSElapsedTime|ActualElapsedTime|AirTime|Distance|CarrierDelay|WeatherDelay|NASDelay|SecurityDelay|LateAircraftDelay|UnusedColumn|DayofWeek|Month|
+----+-------------+------------+------+-----------+----------+-------------+--------------+-------+---------+--------+------+--------------+-----------+------------+---------+----------------+--------+--------------+-----------------+-------+--------+------------+------------

In [8]:
# Removing columns which have only NULL values for all cancelled fligths (events which occur after successful take-off)
# Including these would give an unfair advantage to the model
flight_df = flight_df.drop("UnusedColumn", "LateAircraftDelay", "SecurityDelay", "NASDelay", "WeatherDelay", "CarrierDelay", "AirTime", "ActualElapsedTime", "ArrivalDelay", "ArrivalTime", "TaxiIn", "WheelsOn", "CancellationCode")

In [9]:
flight_df.describe().toPandas()

,summary,UniqueCarrier,FlightNumber,Origin,Destination,CRSDepTime,DepartureTime,DepartureDelay,TaxiOut,WheelsOff,CRSArrivalTime,Cancelled,Diverted,CRSElapsedTime,Distance,DayofWeek,Month
0,count,6429338,6429338,6429338,6429338,6429338,6346471,6346471,6343551,6343551,6429338,6429338,6429338,6429338,6429338,6429338,6429338
1,mean,None,2293.7116343859975,None,None,1319.4466834688112,1326.026277910984,7.598178735867539,16.036132759080836,1350.7975509300707,1496.7219118049168,0.013537630157257247,0.0023832935832584943,129.3446482359459,724.9700084207736,3.9334544240791196,6.403640779190641
2,stddev,None,2046.4974637635953,None,None,457.7529163543435,468.8940114495748,31.764943958809727,10.563825466907772,470.05145211129746,473.31775527788676,0.11556108689381943,0.04876078203599143,69.66364081735983,561.0229006051012,1.9522775240815582,3.337858994294111
3,min,9E,1,ABE,ABE,1,1.0,-96.0,1.0,1.0,1,0.0,0.0,1.0,11.0,1,1
4,max,YV,7829,YUM,YUM,2359,2400.0,2445.0,458.0,2400.0,2400,1.0,1.0,660.0,4962.0,7,12


In [10]:
# Replacing missing values with dummy values as mean/median may be too misleading for the model
flight_df = flight_df.fillna({
    "DepartureTime": -1,
    "DepartureDelay": -999,
    "TaxiOut": -1,
    "WheelsOff": -1
})

### Filtering out diverted flights

In [11]:
flight_df = flight_df.filter(F.col("Diverted") != 1)
flight_df = flight_df.drop("Diverted")

### Result of data cleaning and preprocessing

In [12]:
flight_df.describe().toPandas()

,summary,UniqueCarrier,FlightNumber,Origin,Destination,CRSDepTime,DepartureTime,DepartureDelay,TaxiOut,WheelsOff,CRSArrivalTime,Cancelled,CRSElapsedTime,Distance,DayofWeek,Month
0,count,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015,6414015
1,mean,None,2293.3799955254235,None,None,1319.4701336058615,1308.8618974230649,-5.437568823895797,15.801064543815379,1332.6911000364046,1496.657665284537,0.013569971382979305,129.29364617949912,724.5877844064911,3.9332388527310895,6.4038607642794725
2,stddev,None,2046.2889367772186,None,None,457.77020834105366,489.36860848625486,117.94977839696475,10.659554576419781,492.0425485981101,473.3234123810263,0.11569714493715916,69.6402235039398,560.8469864545418,1.9522640447262647,3.3381621071187064
3,min,9E,1,ABE,ABE,1,-1.0,-999.0,-1.0,-1.0,1,0.0,1.0,11.0,1,1
4,max,YV,7829,YUM,YUM,2359,2400.0,2445.0,458.0,2400.0,2400,1.0,660.0,4962.0,7,12
